In [1]:
import geopandas as gpd
import os
from ecosystools import ecosys_input,Site, Climate, Plant,Site,Soil
from ecosystools.utils import *
from osgeo import ogr
import random
import time
import pickle

In [ ]:
#read variables from config.config
import configparser
import ast
config = configparser.ConfigParser()
config.read('config.fig')

['config_2021_global.config']

In [3]:
###
model_start_year = config['general'].getint('model_start_year')
experiment_start_year = config['general'].getint('experiment_start_year')
experiment_end_year = config['general'].getint('experiment_end_year')

###
NLDAS_database = config['general']['NLDAS_database']
gSSURGO_Dir = config['general']['gSSURGO_Dir']
Global_DEM_file = config['general']['Global_DEM_file']
Global_Ta_file = config['general']['Global_Ta_file']

##
exe_path = config['general']['ecosys_exe']
PFT_dir = config['general']['PFT_dir']
output_define_dir = config['general']['ecosys_output']

In [4]:
dir0 = '/work/hdd/bbkc/yizhi/carbon_input_database/'
###  project dir
prj_dir = config['general']['prj_dir']

###
SampleRuns_Dir = config['Surrogate_model_Dir']['SampleRuns_Dir']
SurroModel_Dir = config['Surrogate_model_Dir']['SurrogateModel_Dir']

if not os.path.exists(SampleRuns_Dir):
    os.makedirs(SampleRuns_Dir,exist_ok=True)

if not os.path.exists(SurroModel_Dir):
    os.makedirs(SurroModel_Dir,exist_ok=True)

In [5]:
Crop_Type_Names = {}
for key in config['CropID']:
    Crop_Type_Names[key]=config['CropID'][key]
    

plant_density = {}
for key in config['plant_density']:
    plant_density[key]=config['plant_density'].getfloat(key)
    
Harvested = {}
for key in config['Harvested']:
    Harvested[key]=config['Harvested'].get(key)


In [6]:
#define temporary outputs 
tmp_Run_Dir = '/dev/shm/tmp_JAMES_run/'
if not os.path.exists(tmp_Run_Dir):
    os.makedirs(tmp_Run_Dir,exist_ok=True)

In [7]:
sel_shp = gpd.read_file('sample0.1field_4surrogate.geojson')
uids=np.array(sel_shp['unique_fid'].values)#np.array(shp.index)
ct_ids =np.array(sel_shp['cty_id'].values)
#  reproject  
centroid_equal_area = sel_shp.to_crs('+proj=cea').centroid.to_crs('EPSG:4326')
field_Lon = centroid_equal_area.x.values
field_Lat = centroid_equal_area.y.values

In [8]:
nsamples = config['Calibrated_parameters'].getint('nsamples')
for k in config['Calibrated_parameters'].keys():
    if k == 'nsamples':
        continue
    tmp = ast.literal_eval(config['Calibrated_parameters'][k])
    problem_={
        'num_vars':len(tmp.keys()),
        'names':list(tmp.keys()),
        'bounds': [tmp[k] for k in tmp.keys()]
    }
    exec('problem_%s=problem_'% k)
    
    #sample the parameters space
    # if exe.is_parent():
    if not os.path.exists(SurroModel_Dir+'param_values_%s.npy' % k):
        param_values_ = lhsmdu_Uniform(eval('problem_%s'% k),nsamples)
        exec('param_values_%s=param_values_'% k)
        np.save(SurroModel_Dir+'param_values_%s.npy' % k,eval('param_values_%s'% k))
    
    time.sleep(1)
    while True:
        try:
            param_values_ = np.load(SurroModel_Dir+'param_values_%s.npy' % k)
            exec('param_values_%s=param_values_'% k)
            break
        except:
            time.sleep(.25)

In [9]:
#get gSSURGO soil dataset
def write_mesoil(work_dir, u_idx, c_idx):
    driver = ogr.GetDriverByName("OpenFileGDB")
    conn = driver.Open(gSSURGO_Dir + 'gSSURGO_CONUS.gdb', 0)
    gSSURGO_MapunitRaster_file = gSSURGO_Dir + 'MapunitRaster_30m.tif'
    mapunits = Soil.get_gSSURGO_mapunit(gSSURGO_MapunitRaster_file,
                                        sel_shp.loc[(sel_shp["unique_fid"]==u_idx) & (sel_shp["cty_id"] == c_idx)])
    if len(mapunits)>0:
        cokeys, cokeys_pct = Soil.get_mapunit_components(conn,mapunits[0])
        if len(cokeys) > 0:
            output_file = work_dir + 'mesoil'
            try:
                for ci in range(len(cokeys)):
                    status = Soil.write_gSSURGO_soil(conn,cokeys[ci],output_file)
                    if not status:
                        print(u_idx,ci,'soil file incorrect')
                        if os.path.exists(output_file):
                            os.system('rm %s' % output_file)
                        continue
                    else:
                        print(u_idx,ci)
                        break

            except Exception as e:
                print(u_idx,e)
    else:
        print(u_idx,'no soil file corrected')   
    conn = None
    if not os.path.exists(output_file):
        return False
    else:
        return True
    

def get_fertilizer(fertilizer_event):
    f_default = {'N_form': 'default',
                'P_form': 'default',
                'fertilizer_method': 'banded',
                'F_depth': 0.02,
                'F_inhibitors': 1,
                'Manure_N_amount': 0,
                'Manure_C_amount': 0}
    
    for k in f_default.keys():
        if k not in fertilizer_event.keys():
            fertilizer_event[k]=f_default[k]
    return fertilizer_event

In [12]:
# check crop window
cw_start_year = crop_windows["start_year"]
tmp_cw = crop_windows["data"][1766]
for k in tmp_cw.keys():
    if (str(tmp_cw[k]) not in Crop_Type_Names.keys()):
        continue
    if (str(tmp_cw[k])=='covercrop'):
        tmp_cw[k]=-1
        
    plant_DOY, harvest_DOY = [int(float(x)) for x in k.split('-')]
    p_year,p_DOY = julian2julian(cw_start_year,plant_DOY)
    h_year,h_DOY = julian2julian(cw_start_year,harvest_DOY)    
    print(p_year,p_DOY,h_year,h_DOY,tmp_cw[k])

NameError: name 'crop_windows' is not defined

In [13]:
sel_shp.head(5)

,unique_fid,cty_id,state_id,geometry
0,1883,19133,19,"POLYGON ((-95.81314 42.15394, -95.81707 42.153..."
1,21254,19133,19,"POLYGON ((-95.9673 42.15382, -95.96231 42.1537..."
2,19111,19133,19,"POLYGON ((-95.84348 42.0602, -95.84271 42.0602..."
3,15835,19133,19,"POLYGON ((-96.08336 42.00873, -96.0833 42.0122..."
4,15329,19133,19,"POLYGON ((-96.00221 41.92145, -96.00202 41.921..."


In [ ]:
def work(task):
    tsk, SR, state = task
    
    u_id =uids[tsk]
    c_id = ct_ids[tsk]

    #crop_windows
    with open(dir0 + "%s/data/final_data_cropland/crop_window/%s.pkl"%(state, c_id), 'rb') as f:
        crop_windows = pickle.load(f)

    #fertilizer
    with open(dir0 + "%s/data/final_data_cropland/fertilizer/%s.pkl"%(state, c_id), 'rb') as f:
        fertilizer = pickle.load(f)
   
    # tillage
    with open(dir0 + "%s/data/final_data_cropland/tillage/%s.pkl"%(state,c_id), 'rb') as f:
        till= pickle.load(f)
        
    lon = field_Lon[np.logical_and(sel_shp["unique_fid"]==u_id, sel_shp["cty_id"] == c_id)][0]
    lat = field_Lat[np.logical_and(sel_shp["unique_fid"]==u_id, sel_shp["cty_id"] == c_id)][0]     
    site_DEM = get_elevation(Global_DEM_file,lon,lat)

    print(u_id, c_id, state)
    
    basename =  '%s_%d_%d' % (c_id, u_id, SR)
    
    work_dir = SampleRuns_Dir + str(basename) + os.sep

    if os.path.exists(work_dir):
        os.system('rm -r "%s"' % work_dir)    
    os.makedirs(work_dir,exist_ok=True)
    
    #extract climate data from NLDAS database
    for year in range(model_start_year,experiment_end_year+1):
        if os.path.exists(work_dir+'me%0.4dw.csv' % year):
            continue
        s=Climate.Climate(TTYPE='H')
        s.read_NLDAS(NLDAS_database,lon,lat,year)
        s.write_climate(work_dir+'me%0.4dw.csv' % year)
    
    #write gSSURGO-based soil file
    status = write_mesoil(work_dir,u_id, c_id)
    if not status:
        return
    
    #extract DEM and multiyear-averaged Ta for mesite setup
    site_DEM = get_elevation(Global_DEM_file,lon,lat)
    site_Ta = get_Ta(Global_Ta_file,lon,lat)

    #define site information
    tmp_site = Site.Site(ALATG=lat,ALTIG=site_DEM,ATCAG=site_Ta,IERSNG=1)

    #initialize the input information
    s = ecosys_input.ecosys_input(start_year=model_start_year,end_year=experiment_end_year,n_cycles=1,
                                mesite = tmp_site,
                                output_start_year=experiment_start_year,model_start_year=model_start_year)
    #update crop, fertilizer, tillage, and irrigation information
    
    if u_id in crop_windows['data'].keys():
        #
        crop_window = crop_windows['data'][u_id]
        for k in crop_window.keys():
            plant_DOY, harvest_DOY = [int(float(x)) for x in k.split('-')]
            crop_type = Crop_Type_Names[str(int(float(crop_window[k])))]
            p_year,p_month,p_mday = julian2calendar(crop_windows['start_year'],plant_DOY)
            h_year,h_month,h_mday = julian2calendar(crop_windows['start_year'],harvest_DOY)
            p_density = plant_density[crop_type]   

    
            if crop_type not in Harvested.keys() or Harvested[crop_type]=='NO':
                if p_year%2==0:
                    s.Planting.add_covercrop(planting_year=p_year, planting_month=p_month, planting_mday=p_mday, crop_type=crop_type+'0',
                                            PPI=p_density, terminate_year=h_year, terminate_month=h_month, terminate_mday=h_mday)    
                else:
                    s.Planting.add_covercrop(planting_year=p_year, planting_month=p_month, planting_mday=p_mday, crop_type=crop_type+'1',
                                            PPI=p_density, terminate_year=h_year, terminate_month=h_month, terminate_mday=h_mday)                   
            else:
                if p_year%2==0:
                    s.Planting.add_cashcrop(planting_year=p_year, planting_month=p_month, planting_mday=p_mday, crop_type=crop_type+'0',
                                            PPI=p_density, terminate_year=h_year, terminate_month=h_month, terminate_mday=h_mday) 
                else:
                    s.Planting.add_cashcrop(planting_year=p_year, planting_month=p_month, planting_mday=p_mday, crop_type=crop_type+'1',
                                            PPI=p_density, terminate_year=h_year, terminate_month=h_month, terminate_mday=h_mday)                         
                
    if u_id in fertilizer['data'].keys():
        fertilizer_events = fertilizer['data'][u_id]
        for k in fertilizer_events.keys():
            f_DOY = int(k)
            f_year,f_month,f_mday = julian2calendar(fertilizer['start_year'],f_DOY)
            
            fertilizer_event = get_fertilizer(fertilizer_events[k]) # complete the fertilizer information with default values
            s.Fertilizer.add_by_type(year = f_year, month = f_month, mday = f_mday, 
                                    N_rate = fertilizer_event['N_rate']*0.112085116, P_rate = fertilizer_event['P_rate']*0.112085116, 
                                    N_form = fertilizer_event['N_form'], P_form = fertilizer_event['P_form'], 
                                    fertilizer_method = fertilizer_event['fertilizer_method'], F_depth = fertilizer_event['F_depth'],  
                                    F_inhibitors = fertilizer_event['F_inhibitors'], Manure_N_amount = fertilizer_event['Manure_N_amount'],
                                    Manure_C_amount = fertilizer_event['Manure_C_amount'])
    #tillage 
    if u_id in till['data'].keys():
        till_events = till['data'][u_id]
        for k in till_events.keys():
            t_DOY = int(k)
            t_year,t_DOY = julian2julian(till['start_year'],t_DOY)
            if t_year < experiment_start_year or t_year > experiment_end_year:
                continue
            
            if t_DOY == 366:
                t_DOY=365            
                
            till_info = till_events[k] # 

            t_year,t_month,t_mday = julian2calendar(t_year, t_DOY)
        
            s.Disturbance.add(year=t_year,month=t_month,mday=t_mday,
                                    IPLOW=till_info['mixing_ratio'],DPLOW=till_info['tillage_depth'])
 
    s.write_runfile(run_dir=work_dir)

    #run ecosys model 
    os.system('cp  "{}"/sc*1 "{}"'.format(output_define_dir, work_dir))   # define outputfiles

    #revise the crop parameters
    
    for v in Crop_Type_Names.values():
        if v not in config['Calibrated_parameters'].keys():
            os.system('cp  "{}" "{}0"'.format(PFT_dir+v, work_dir+v)) 
            os.system('cp  "{}" "{}1"'.format(PFT_dir+v, work_dir+v)) 

    for k in config['Calibrated_parameters'].keys():
        if k == 'nsamples':
            continue
        p_ = Plant.Plant_Species(PFT_dir+k)
        p_.revise_parameters(eval('problem_%s' % k)['names'], eval('param_values_%s' % k)[SR,:], work_dir + k+'0')
        p_.revise_parameters(eval('problem_%s' % k)['names'], eval('param_values_%s' % k)[SR,:], work_dir + k+'1')
    
    run_ecosys(exe_path,work_dir,runfile='runfile',runlog='runlog.log',rm_work_dir=False,output_db=SampleRuns_Dir+basename+'.db')

In [11]:
state_name = {"17": "IL", "18": "IN", "19":"IA","20":"KS", "26":"MI", "27":"MN","29": "MO","38":"ND",
              "31" :"NE","39": "OH","46": "SD","55": "WI"}

In [12]:
tasks = []    
for SR in range(40):
    fi = random.randint(0, len(sel_shp)-1)
    county_id = sel_shp["cty_id"].values[fi]
    state_id = county_id[0:2]

    tasks.append((fi, SR, state_name[state_id]))

In [13]:
import time

In [ ]:
t=time.time()
for i in range(20,40):
    work(tasks[i])

print(time.time()-t)

9103 31081 NE
9103 0
scch1
scwh1
sceh1
scph1
scnh1
sccd1
scwd1
sced1
scpd1
scnd1
8643 46079 SD
8643 0
scch1
scwh1
sceh1
scph1
scnh1
sccd1
scwd1
sced1
scpd1
scnd1
26081 20013 KS
26081 0
